# Titanic - Machine Learning from Disaster
## Start here! Predict survival on the Titanic and get familiar with ML basics

* Utilizando os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic) para tentar prever a porcentagem de passageiros que sobreviveram ao desastre. A intenção é obter a maior acurácia possível.

In [1]:
# Importando o Pandas
import pandas as pd

### Entendendo a base

In [2]:
# Visualizando a base de treino
treino = pd.read_csv('train.csv')
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* **Utilizaremos o ydata-profiling (antigo Pandas Profiling) para entender melhor a base**:
    * https://github.com/ydataai/ydata-profiling

In [3]:
# Instalando o ydata-profiling
pip install ydata-profiling

SyntaxError: invalid syntax (177699198.py, line 2)

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
# Gerando a visualização
profile = ProfileReport(treino, title="titanic_treino")

In [4]:
# Criando o relatório
profile.to_file("titanic_treino.html")

NameError: name 'profile' is not defined

* **Vamos analisar essa base**:

In [ ]:
# Verificando as informações da base
treino.info()

In [ ]:
# Verificando o tipo dos dados
treino.dtypes.value_counts()

In [5]:
# Valores nulos
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

* Vamos analisar a base de teste:

In [6]:
# Visualizando a base de teste
teste = pd.read_csv('test.csv')
teste.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [7]:
# Verificando as informações da base
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [8]:
# Valores nulos
teste.isnull().sum().sort_values(ascending=False).head(5)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

* Temos colunas que possuem valores vazios na base de teste que não estão vazias na base de treino

## Realizando os tratamentos iniciais dos dados

* **Começamos tratando os valores vazios**:

In [9]:
# Verificando novamente os valores vazios para a base de treino
# Valores nulos
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

In [10]:
# Retornando o shape da base
treino.shape

(891, 12)

In [11]:
# Visualizando novamente a base
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
# Verificando a cardinalidade dos dados
treino.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

* Colunas com **alta cardinalidade** como **Name**, por exemplo, podem tornar o modelo **menos generalizável**, por isso escolhi **eliminar** essas colunas.

In [13]:
# Eliminando para a base de treino
treino = treino.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [14]:
# Eliminando para a base de teste
teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis=1)

* Precisamos tratar a coluna **Age**.
*  Para isso, vamos utilizar a **média** das idades para substituir os **valores nulos**.

In [24]:
# Verficiando a média das idades para a base de treino
treino.Age.mean()

29.69911764705882

In [27]:
# Selecionar as linhas com a idade vazia e substituir pela média
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean()

In [28]:
# Fazer o mesmo apra a base de teste
teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean()

* Para a base de treino, ainda temos 2 valores vazios na coluna **Embarked**.
*  Vamos substituir esses valores pela **moda**.

In [30]:
# Verificando os valores na coluna Embarked
treino.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [32]:
# Selecionando a moda dessa coluna
treino.Embarked.mode()[0]

'S'

In [33]:
# Substituindo os valores vazios
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

In [34]:
# Substituindo os valores nulos da coluna Fare pela média
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

* Verifiando novamente os valores vazios

In [35]:
# Base de treino
treino.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

In [36]:
# Base de teste
teste.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

In [37]:
# Verificando as colunas de texto na base de treino
treino.columns[treino.dtypes != 'object']

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [39]:
# Selecionando apenas os valores numéricos da base de treino
treino_nr = treino.loc[:, treino.columns[treino.dtypes != 'object']]
treino_nr

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.000000,1,0,7.2500
1,2,1,1,38.000000,1,0,71.2833
2,3,1,3,26.000000,0,0,7.9250
3,4,1,1,35.000000,1,0,53.1000
4,5,0,3,35.000000,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,0,2,27.000000,0,0,13.0000
887,888,1,1,19.000000,0,0,30.0000
888,889,0,3,29.699118,1,2,23.4500
889,890,1,1,26.000000,0,0,30.0000


In [40]:
# O mesmo para a base de teste
teste_nr = teste.loc[:, teste.columns[teste.dtypes != 'object']]
teste_nr

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.50000,0,0,7.8292
1,893,3,47.00000,1,0,7.0000
2,894,2,62.00000,0,0,9.6875
3,895,3,27.00000,0,0,8.6625
4,896,3,22.00000,1,1,12.2875
...,...,...,...,...,...,...
413,1305,3,30.27259,0,0,8.0500
414,1306,1,39.00000,0,0,108.9000
415,1307,3,38.50000,0,0,7.2500
416,1308,3,30.27259,0,0,8.0500


## Podemos selecionar agora um modelo para classificar esses dados

* Para começar, vamos testar entre:

 *  **Árvore de classificação**
   *   https://scikit-learn.org/stable/modules/tree.html#classification
  *  **Classificação dos vizinhos mais próximos**
   *   https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
  *  **Regressão Logística**
   *   https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

* Antes, precisamos separar a base de treino em **treino e validação**.
*  Vamos fazer isso utilizando o train_test_split.

In [41]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [43]:
# Separando a base de treino em x e y
X = treino_nr.drop(['PassengerId', 'Survived'], axis=1)
y = treino.Survived

In [44]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.33, random_state=42)

* Para a **Árvore de Classificação**:

In [46]:
from sklearn import tree

In [49]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [50]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train, y_train)

In [57]:
# Fazendo a validação
y_pred_ac = clf_ac.predict(X_val)

* Para o **KNeighborsClassifier**:

In [52]:
# Importando 
from sklearn.neighbors import KNeighborsClassifier

In [60]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [61]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train, y_train)

In [62]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

C:\Users\gabri\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


* Para a **Regressão Logística**:

In [64]:
# Importando
from sklearn.linear_model import LogisticRegression

In [65]:
# Criando o classificador
clf_rl = LogisticRegression(random_state = 42)

In [66]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train, y_train)

In [67]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

## Vamos avaliar esses modelos

* Para fazer a avaliação, podemos utilizar:
  * Acurácia (método de avaliação usado na competição):
   * https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
  * Matriz de confusão (ajuda a visualizar a distribuição dos erros):
   * https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html


* Avaliando a **acurácia**:

In [68]:
# Importando
from sklearn.metrics import accuracy_score

In [69]:
# Para a árvore
accuracy_score(y_val, y_pred_ac)

0.6169491525423729

In [70]:
# Para o knn
accuracy_score(y_val, y_pred_knn)

0.6542372881355932

In [71]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

0.7254237288135593

* Avaliando a **matriz de confusão**:

In [72]:
# Importando
from sklearn.metrics import confusion_matrix

In [73]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[125,  50],
       [ 63,  57]], dtype=int64)

In [74]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

array([[133,  42],
       [ 60,  60]], dtype=int64)

In [75]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[156,  19],
       [ 62,  58]], dtype=int64)

## Fazendo a previsão para os dados de teste

* Vamos usar o modelo com melhor precisão para fazer o predict na base de teste.

In [76]:
# Visualizando o X_train
X_train

,Pclass,Age,SibSp,Parch,Fare
6,1,54.000000,0,0,51.8625
718,3,29.699118,0,0,15.5000
685,2,25.000000,1,2,41.5792
73,3,26.000000,1,0,14.4542
882,3,22.000000,0,0,10.5167
...,...,...,...,...,...
106,3,21.000000,0,0,7.6500
270,1,29.699118,0,0,31.0000
860,3,41.000000,2,0,14.1083
435,1,14.000000,1,2,120.0000


In [77]:
# Visualizando a base de teste (apenas com valores numéricos)
teste_nr.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.5,0,0,7.8292
1,893,3,47.0,1,0,7.0000
2,894,2,62.0,0,0,9.6875
3,895,3,27.0,0,0,8.6625
4,896,3,22.0,1,1,12.2875


In [78]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste_nr.drop('PassengerId', axis=1)

In [79]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)

In [82]:
# Criando uma nova coluna com a previsão da base de teste
teste['Survived'] = y_pred

In [83]:
# Selecionando apenas a coluna de Id e SUrvived para fazer o envio
base_envio = teste[['PassengerId', 'Survived']]

In [85]:
base_envio.to_csv('resultados.csv', index=False)

In [86]:
res = pd.read_csv('resultados.csv')